In [36]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset, random_split
import pandas as pd
import json
from sklearn.metrics import accuracy_score, classification_report, f1_score

from datasets import load_dataset
from collections import Counter


torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

def load_data():
    data = load_dataset("mila-ai4h/biasly-data", "biasly-data")
    data = data["train"]
    data_hash = {}
    
    non_majority_label_indexes = []
    for i in range(10000):

    #add majority label here and remove those keys that don't have a majority label. 
        if (data[i*3]['is_misogynistic'] == "Unclear" or 
            data[i*3+1]['is_misogynistic'] == "Unclear" or 
            data[i*3+2]['is_misogynistic'] == "Unclear"):
            continue
        else:
            labels = [data[i*3]['is_misogynistic'],data[i*3+1]['is_misogynistic'],data[i*3+2]['is_misogynistic']]
            counter = Counter(labels)
            most_common_labels = counter.most_common()
            max_count = most_common_labels[0][1]
            
            # Get all labels with the max count (in case of a tie)
            majority_labels = [label for label, count in most_common_labels if count == max_count]
            data_hash[data[i*3]['datapoint_id']] = {
                'datapoint_id': data[i*3]['datapoint_id'], 
                'majority':  majority_labels[0],
                'text': data[i*3]['datapoint'],
                'annotators': [
                {'annotator_id': data[i*3]['annotator_id'], 'is_misogynistic': data[i*3]['is_misogynistic']},
                {'annotator_id': data[i*3+1]['annotator_id'], 'is_misogynistic': data[i*3+1]['is_misogynistic']},
                {'annotator_id': data[i*3+2]['annotator_id'], 'is_misogynistic': data[i*3+2]['is_misogynistic']}
            ]
            }
    
    return data_hash


# Load the HateXplain dataset
def load_hatexplain():
    data_hash = load_data()
    
    train_texts, train_labels, val_texts, val_labels, test_texts, test_labels = [], [], [], [], [], []
    label_map = {"Yes": 1, "No": 0}

    train_data = dict(list(data_hash.items())[:7935])      # First 7935 elements
    val_data = dict(list(data_hash.items())[7935:8935])     # Next 1000 elements
    test_data = dict(list(data_hash.items())[8935:])        # Last 1000 elements
    
    for key, value in train_data.items():
        train_texts.append(value['text'])
        train_labels.append(label_map[value["majority"]])
        
    for key, value in val_data.items():
        val_texts.append(value['text'])
        val_labels.append(label_map[value["majority"]])

    for key, value in test_data.items():
        test_texts.append(value['text'])
        test_labels.append(label_map[value["majority"]])
        
    return train_texts, val_texts, test_texts, train_labels, val_labels, test_labels
    
# Custom PyTorch dataset class
class HateSpeechDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }
train_texts, val_texts, test_texts, train_labels, val_labels, test_labels = load_hatexplain()


# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Create PyTorch datasets
train_dataset = HateSpeechDataset(train_texts, train_labels, tokenizer)
val_dataset = HateSpeechDataset(val_texts, val_labels, tokenizer)
test_dataset = HateSpeechDataset(test_texts, test_labels, tokenizer)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


cpu


In [ ]:

# Load BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

# Optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

from tqdm import tqdm

# Training loop with progress bar, accuracy, and F1-score
def train_model(model, train_loader, val_loader, epochs=3, save_dir="saved_models"):
    best_val_acc = 0  # Track highest validation accuracy
    best_model_path = None
    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        all_preds, all_labels = [], []

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        for batch in progress_bar:
            input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()

            # Compute training loss and accuracy
            total_loss += loss.item()
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            # Store predictions and labels for F1-score
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Update progress bar
            progress_bar.set_postfix(loss=loss.item())

        # Compute metrics
        train_acc = correct / total
        train_f1 = f1_score(all_labels, all_preds, average='macro')

        # Validation evaluation
        val_acc, val_f1 = evaluate(model, val_loader)

        # Save model for this epoch
        model_path = os.path.join(save_dir, f"model_epoch_{epoch+1}.pt")
        torch.save(model.state_dict(), model_path)
        
        # Track the best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_path = model_path  # Save best model path

        print(f"Epoch {epoch+1}: Train Loss: {total_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")
    print(f"Best model saved at {best_model_path} with Val Acc: {best_val_acc:.4f}")
    last_model_path = os.path.join(save_dir, f"model_epoch_{epochs}.pt")
    return best_model_path, last_model_path


# Evaluation function with accuracy and F1-score
def evaluate(model, data_loader, final=False):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in data_loader:
            input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)
            # print(tokenizer.decode(input_ids[0])) to check if it's in order
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = outputs.logits.argmax(dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    macro_f1 = f1_score(all_labels, all_preds, average='macro')

    if final:
        print("\nFinal Test Classification Report:\n", classification_report(all_labels, all_preds))

    return accuracy, macro_f1

# Train the model
best_model_path, last_model_path = train_model(model, train_loader, val_loader, epochs=3)


# Evaluate on test set after training best model
print("\nFinal evaluation on test set best model:")
model.load_state_dict(torch.load(best_model_path))
test_acc, test_f1 = evaluate(model, test_loader, final=True)
print(f"Test Accuracy: {test_acc:.4f}, Test Macro F1-score: {test_f1:.4f}")

# Evaluate on test set after training last model
print("\nFinal evaluation on test set last model:")
model.load_state_dict(torch.load(last_model_path))
test_acc, test_f1 = evaluate(model, test_loader, final=True)
print(f"Test Accuracy: {test_acc:.4f}, Test Macro F1-score: {test_f1:.4f}")


In [56]:
len(data_hash.keys())

9935

In [44]:
count_no_majority = sum(1 for entry in data_hash.values() if entry['majority'] == "No")
print(count_no_majority)


8293


In [58]:
len(dict(list(data_hash.items())[7935:8935]) )

1000

In [12]:
ds["train"][0]

{'Unnamed: 0': 0,
 'datapoint_id': '9dd77afa-dfc8-4adb-87b2-b51f72ccc5fc',
 'datapoint': "ititmeansthatit's whatdoesthatmean? ititmeansthatit's really hard to find out where ititmeansthatit's really hard to find out where she was chatting from. Really hard to find out where she was chatting from.",
 'is_misogynistic': 'No',
 'why_unclear': None,
 'misogynistic_inferences': None,
 'other_inferences': None,
 'inferences_explanation': None,
 'original_severity': None,
 'rewrite_possible': None,
 'rewrite': None,
 'rewrite_severity': None,
 'annotator_id': 'A1',
 'annotator_background': 'Linguistics'}

In [ ]:
# 80/10/10 split 
# mum input sequence length of 512, batch size of 32, a learning rate of 2e-5, and 3 epochs for training.
# Can put datapoint id in a 